<a href="https://colab.research.google.com/github/honomax/gradio-wordcloud-app/blob/main/%E3%83%AF%E3%83%BC%E3%83%89%E3%82%AF%E3%83%A9%E3%82%A6%E3%83%89%E7%94%9F%E6%88%90%E3%82%A2%E3%83%97%E3%83%AA(Gradio).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 日本語フォントのインストール

In [ ]:
!apt-get -q install fonts-ipafont-gothic

# ライブラリのインポート

In [ ]:
import io #ファイルやバッファの入出力操作
import chardet #バイナリデータの文字エンコーディングを自動検出
import asyncio #非同期処理（async/await）を実現
import gradio as gr #ウェブUIを作成
from PIL import Image #画像処理用
from wordcloud import WordCloud #テキストからワードクラウド画像を生成
import matplotlib.pyplot as plt #グラフ描画

# ワードクラウド作成

In [ ]:
def generate_wordcloud(file):
    try:
        if file is None:
            return None

        with open(file.name, 'rb') as f:
            rawdata = f.read()

        result = chardet.detect(rawdata)
        encoding = result['encoding'] if result['encoding'] else 'utf-8'
        text = rawdata.decode(encoding)

        FONT_PATH = "/usr/share/fonts/truetype/fonts-ipafont-gothic/ipagp.ttf"

        wordcloud = WordCloud(
            font_path=FONT_PATH,
            width=800,
            height=400,
            background_color='white',
            random_state=42  # 再現性のための固定seed
        ).generate(text)

        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.tight_layout(pad=0)

        img_buf = io.BytesIO()
        plt.savefig(img_buf, format='png')
        plt.close()
        img_buf.seek(0)

        img = Image.open(img_buf)

        return img

    except Exception as e:
        import traceback
        error_msg = f"エラーが発生しました:\n{str(e)}\n\n{traceback.format_exc()}"
        return None, error_msg

# アプリの実行

In [ ]:
iface = gr.Interface(
    fn=generate_wordcloud,
    inputs=gr.File(label="テキストファイルをアップロードしてください", file_types=['.txt']),
    outputs=gr.Image(type="pil", label="ワードクラウド"),
    title="ワードクラウド生成アプリ",
    description="テキストファイルをアップロードするとワードクラウドを生成して表示します。"
)

if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    iface.launch()